In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import datetime
from pyspark.sql.functions import col, lit, avg
from pyspark.sql import *

In [2]:
#importing csv file from hdfs
user_data = sc.textFile("hdfs://localhost:54310/user/hive/warehouse/data/user_info.csv").map(lambda x:x.split(","))

In [3]:
user_df = sqlContext.createDataFrame(data=user_data.filter(lambda x:x[0]!='user_name'),
                             schema=user_data.filter(lambda x:x[0]=='user_name').collect()[0])

In [4]:
# creating new column named seconds with username
hours_list = []
for row in user_df.collect():
    # converting string type of datetime to datetime
    date_time = datetime.datetime.strptime(row['idle_time'], "%Y-%m-%d %H:%M:%S")
    seconds_data = date_time.hour * 3600 + date_time.minute *60 + date_time.second
    hours_list.append((row['user_name'], seconds_data))

In [5]:
#creating new column dataframe to join with original dataframe( new_log_dataframe)
column = sqlContext.createDataFrame(hours_list, ('username',"seconds"))

In [6]:
#merge new column with the dataframe 
dataframe = (user_df
.join(column, col("user_name")==col("username")))


In [7]:
#obtaining avarage value of seconds by using aggregate function
average_data=dataframe.agg(avg(col("seconds")))
average = average_data.collect()[0][0]

In [8]:
#filtering dataframe with highest average hours users
highest_idle_hours = dataframe.filter(dataframe['seconds'] > average)


In [9]:
highest_idle_hours.select("user_name","seconds").show()

+--------------------+-------+
|           user_name|seconds|
+--------------------+-------+
|       nikitapawar17|  22800|
|“shivnajalisangal...|  35400|
|gaikwadr576@gmail...|  11100|
|magadum.iranna@gm...|   8100|
| dileep.bs@yahoo.com|  12300|
|puruissimple@gmai...|  11700|
|hakepratiksha55@g...|  33900|
|tekina.makin@gmai...|  33900|
| addyp1911@gmail.com|  11400|
| blsonalib@gmail.com|   9300|
|20150773@dbatu.ac.in|  11100|
|farooqbassam4@gma...|   9000|
|ruchikachile30199...|  12900|
|polelaxman001@gma...|  33300|
|ayush.saraf47@gma...|  15600|
|surajpj7852@gmail...|   7800|
|     you@example.com|   9300|
|vishnu23kumar@gma...|  15300|
|  sahil24c@gmail.com|  18300|
|sargampandey27oct...|  17100|
+--------------------+-------+
only showing top 20 rows



In [11]:
#filtering users having highest idle value
highest_idle_hours.select("user_name").show()

+--------------------+
|           user_name|
+--------------------+
|       nikitapawar17|
|“shivnajalisangal...|
|gaikwadr576@gmail...|
|magadum.iranna@gm...|
| dileep.bs@yahoo.com|
|puruissimple@gmai...|
|hakepratiksha55@g...|
|tekina.makin@gmai...|
| addyp1911@gmail.com|
| blsonalib@gmail.com|
|20150773@dbatu.ac.in|
|farooqbassam4@gma...|
|ruchikachile30199...|
|polelaxman001@gma...|
|ayush.saraf47@gma...|
|surajpj7852@gmail...|
|     you@example.com|
|vishnu23kumar@gma...|
|  sahil24c@gmail.com|
|sargampandey27oct...|
+--------------------+
only showing top 20 rows

